<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/separate_gwn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [2]:
import pandas as pd
import numpy as np

from ddop.newsvendor import GaussianWeightedNewsvendor
from ddop.newsvendor import SampleAverageApproximationNewsvendor

from sklearn.model_selection import RandomizedSearchCV

from ddop.metrics import make_scorer
from ddop.metrics import average_costs
from ddop.metrics import prescriptiveness_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## loading data-set


---



*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**

In [4]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

## split in feature and target matrix


---



In [5]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split


---



In [6]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

## defining the different cost paramerters for special service level and product range



---



*   cu is the mean of the cumulated 'sell_price' column of the training data
*   appending to the corresponding service level co is calculated with (cu - α * cu) / α
*   all the costs are rounded to 2 digits

In [7]:
prods = list(range(0,25))

cu = round(X_train['sell_price'].mean(),2)

co_90 = round(((cu-(0.90*cu))/0.90),2)

co_75 = round(((cu-(0.75*cu))/0.75),2)

co_50 = round(((cu-(0.50*cu))/0.50),2)

## determining some parameters


---


*   the potential range of the params for randomized search are defined
*   the estimator, the scorer and finally the whole param_random is defined

In [8]:
kernel_bandwidth = np.arange(25, 81)
param_random = dict(kernel_bandwidth = kernel_bandwidth)

scorer_avc = make_scorer(average_costs, greater_is_better=False)

# gwn for 90% service level


---



## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,9
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_kbwth_90 = []

for prod in prods:
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_90)

  random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  kbwth = random.best_params_.get('kernel_bandwidth')

  best_param_kbwth_90.append(kbwth)

## newsvendor-model


---


*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be calculated)
*   save the results as pandas frame



In [26]:
result_gwn_90_avc = []
for prod in prods:
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_90, kernel_bandwidth = best_param_kbwth_90[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = gwn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_90),2)
  result_gwn_90_avc.append(avc)

In [28]:
GWN_separate_avc_90 = pd.DataFrame(result_gwn_90_avc, columns=['GWN_avc_90'])
GWN_separate_avc_90.index.name = 'item_id'

In [30]:
result_gwn_90_pscr = []
for prod in prods:
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_90, kernel_bandwidth = best_param_kbwth_90[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_gwn = gwn.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_90)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_gwn, preds_saa, cu = cu, co = co_90)
  result_gwn_90_pscr.append(pscr)

In [33]:
GWN_separate_pscr_90 = pd.DataFrame(result_gwn_90_pscr, columns=['GWN_pscr_90'])
GWN_separate_pscr_90.index.name = 'item_id'

# gwn for 75% service level


---



## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,75
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_kbwth_75 = []

for prod in prods:
  
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_75)

  random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  kbwth = random.best_params_.get('kernel_bandwidth')

  best_param_kbwth_75.append(kbwth)

## newsvendor-model


---


*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be calculated)
*   save the results as pandas frame

In [36]:
result_gwn_75_avc = []
for prod in prods:
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_75, kernel_bandwidth = best_param_kbwth_75[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = gwn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_75),2)
  result_gwn_75_avc.append(avc)

In [37]:
GWN_separate_avc_75 = pd.DataFrame(result_gwn_75_avc, columns=['GWN_avc_75'])
GWN_separate_avc_75.index.name = 'item_id'

In [38]:
result_gwn_75_pscr = []
for prod in prods:
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_75, kernel_bandwidth = best_param_kbwth_75[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_gwn = gwn.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_75)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_gwn, preds_saa, cu = cu, co = co_75)
  result_gwn_75_pscr.append(pscr)

In [39]:
GWN_separate_pscr_75 = pd.DataFrame(result_gwn_75_pscr, columns=['GWN_pscr_75'])
GWN_separate_pscr_75.index.name = 'item_id'

# gwn for 50% service level


---



## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,5
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_kbwth_50 = []

for prod in prods:
  
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_50)

  random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  kbwth = random.best_params_.get('kernel_bandwidth')

  best_param_kbwth_50.append(kbwth)

## newsvendor-model


---


*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be calculated)
*   save the results as pandas frame

In [48]:
result_gwn_50_avc = []
for prod in prods:
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_50, kernel_bandwidth = best_param_kbwth_50[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = gwn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_50),2)
  result_gwn_50_avc.append(avc)

In [49]:
GWN_separate_avc_50 = pd.DataFrame(result_gwn_50_avc, columns=['GWN_avc_50'])
GWN_separate_avc_50.index.name = 'item_id'

In [50]:
result_gwn_50_pscr = []
for prod in prods:
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_50, kernel_bandwidth = best_param_kbwth_50[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_gwn = gwn.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_50)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_gwn, preds_saa, cu = cu, co = co_50)
  result_gwn_50_pscr.append(pscr)

In [51]:
GWN_separate_pscr_50 = pd.DataFrame(result_gwn_50_pscr, columns=['GWN_pscr_50'])
GWN_separate_pscr_50.index.name = 'item_id'

# merge the pandas frames


---

In [53]:
GWN_separated_avc = pd.concat([GWN_separate_avc_90, GWN_separate_avc_75, GWN_separate_avc_50], axis=1)
GWN_separated_avc

,GWN_avc_90,GWN_avc_75,GWN_avc_50
item_id,,,
0,2.56,5.48,9.46
1,1.40,2.69,4.36
2,2.50,4.94,8.36
3,2.48,4.68,8.15
4,2.34,5.08,9.52
5,1.66,3.16,5.61
6,1.89,4.18,7.28
7,1.88,3.85,7.13
8,1.46,2.92,5.30


In [56]:
GWN_separated_pscr = pd.concat([GWN_separate_pscr_90, GWN_separate_pscr_75, GWN_separate_pscr_50], axis=1)
GWN_separated_pscr

,GWN_pscr_90,GWN_pscr_75,GWN_pscr_50
item_id,,,
0,0.315403,0.268284,0.248668
1,0.221506,0.316797,0.308511
2,0.143769,0.120050,0.097693
3,0.531595,0.539255,0.470732
4,0.276378,0.179394,0.145331
5,0.203007,0.250993,0.124837
6,0.433397,0.411024,0.388732
7,0.312904,0.198429,0.111343
8,0.247239,0.212590,0.172662


# save the results


---

In [55]:
GWN_separated_avc.to_csv('/content/drive/MyDrive/M5/GWN_separated_avc.csv')

GWN_separated_pscr.to_csv('/content/drive/MyDrive/M5/GWN_separated_pscr.csv')